## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
np.random.seed(seed=33)

# input/output dimensions
input_dims = (1,1,5,5) # BxCxWxH
n_outputs = 5

# model parameters
weights = (np.random.rand(n_outputs,np.product(input_dims[1:])) - 0.5)
biases = (np.random.rand(n_outputs) - 0.5)
inputs = np.random.rand(*input_dims) / 2

# build pyTorch model for comparison
pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(input_dims[1:]), out_features=n_outputs), 
    nn.ReLU(),
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor((biases)))
pt_model(torch.tensor(inputs.reshape(input_dims[0], -1))).detach().numpy()

In [ ]:
# SNN model
t_max = 2**7 # number of time steps to encode value of 1
loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Dense(weights=weights, biases=biases, rectifying=False),
])

# optional neuron probes
units = [quartz.probe(neuron) for neuron in loihi_model.layers[1].output_neurons]

loihi_model(inputs)

In [ ]:
loihi_model

In [ ]:
# optional plotting for each output unit
# [unit.plot() for unit in units]

In [ ]:
%debug

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
np.random.seed(seed=33)

# input / weight dimensions
input_dims = (1,4,7,7) # BxCxWxH
weight_dims = (2,4,3,3) # CoutxCinxWxH
stride = 2
padding = 0
groups = 1
kernel_size = weight_dims[2:]

weights = (np.random.rand(*weight_dims)-0.5) / 2
biases = (np.random.rand(weight_dims[0])-0.5)
inputs = np.random.rand(*input_dims) / 2

# build pyTorch model for comparison
model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1]*groups, out_channels=weight_dims[0], kernel_size=kernel_size, stride=stride, padding=padding, groups=groups), 
    nn.ReLU()
)
model[0].weight = torch.nn.Parameter(torch.tensor(weights))
model[0].bias = torch.nn.Parameter(torch.tensor((biases)))
model(torch.tensor(inputs)).detach().numpy()

In [ ]:
# SNN model
t_max = 2**7 # number of time steps to encode value of 1
loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights, biases=biases, stride=stride, padding=padding, groups=groups),
])

# optional neuron probes
units = [quartz.probe(neuron) for neuron in loihi_model.layers[1].output_neurons]

loihi_model(inputs)

In [ ]:
# optional plotting for each output unit
# [unit.plot() for unit in units]

## Maxpool2d layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
np.random.seed(seed=27)

# define input dimensions and maxpooling kernel size
input_dims = (1,2,4,4)
kernel_size = [2,2]
t_max = 2**7 # number of time steps to encode value of 1

inputs = np.random.rand(*input_dims) / 2
q_inputs = quartz.utils.quantize_inputs(inputs, t_max)

# build pyTorch model for comparison
model = nn.Sequential(
    nn.MaxPool2d(kernel_size=kernel_size),
    nn.ReLU(),
)
model(torch.tensor(q_inputs)).detach().numpy()

In [ ]:
# SNN model
loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.MaxPool2D(kernel_size=kernel_size),
])

wtas = [quartz.probe(neuron) for neuron in loihi_model.layers[1].output_neurons]

loihi_model(inputs)

In [ ]:
# optional plotting for each output unit
# [wta.plot() for wta in wtas]